# Cox-Ross-Rubinstein Model

In [2]:
from math import sqrt, exp
import pydotplus as pydot

def crr_call(s0, k, rate, sigma, n_periods, diagram_out = True):
    import numpy as np
    up_factor = exp(sigma * sqrt(n_periods))
    down_factor = 1/up_factor
    p = ((1+rate) - down_factor)/(up_factor - down_factor)
    # Option Payoff
    contingent_payoff = lambda st, k: max(st - k, 0)
    contingent_payoff = np.vectorize(contingent_payoff)

    # Creating the branches at different times of the tree. Each branch is the possible martingale price
    # of the stock at time t
    stock_price_tree = []
    for i in range(1,n_periods+1):
        # Number of prices per period
        period_prices = []
        for j in range(i+1):
            up_factor_repeat = i-j
            down_factor_repeat = j

            values = np.append(np.repeat(up_factor, up_factor_repeat),
                               np.repeat(down_factor, down_factor_repeat))
            
            stock_price = values.prod() * s0
            period_prices.append(stock_price)
        stock_price_tree.append(period_prices)

    # Getting the payoff of the call at the end of the tree
    payoffs = contingent_payoff(stock_price_tree[n_periods-1], k).tolist()

    option_prices = [payoffs]
    # Contingent option price at period n-1
    branch_n = payoffs # brach at n-1

    for t in range(n_periods):
        branch_nm1 = []
        for i in range(len(branch_n)-1):
            #cu: Contingent price at time n if price rises
            #cd: Contingent price at time n if price decreases
            cu, cd = np.array(branch_n)[[i, i+1]]
            price = exp(-rate)*(p*cu + (1-p)*cd)
            branch_nm1.append(price)
        option_prices.append(branch_nm1)
        branch_n = branch_nm1

    option_prices = option_prices[::-1] 
    stock_price_tree.insert(0, [s0])
    
    # Save the diagram in the current directory if specified
    if diagram_out:
        graph = pydot.Dot(graph_type='digraph', rankdir='LR')

        branch = 0
        counter = 1
        for i, prices in enumerate(option_prices[:-1]):
            next_prices = option_prices[i+1]
            for np, c_price in enumerate(prices):
                c_price = str(round(c_price,2))

                nprice1 = str(round(next_prices[np], 2))
                nprice2 = str(round(next_prices[np+1], 2))

                increment = len(prices)-1
                from_branch, to_branch1, to_branch2 = branch, counter + increment, counter + increment + 1
                
                up_price = round(stock_price_tree[i][np] * up_factor, 2)
                down_price = round(stock_price_tree[i][np] * down_factor, 2)
                edge = pydot.Edge(from_branch, to_branch1, label = str(up_price)); graph.add_edge(edge)
                edge = pydot.Edge(from_branch, to_branch2, label = str(down_price)); graph.add_edge(edge)

                node = pydot.Node(to_branch1, label = str(nprice1)); graph.add_node(node)
                node = pydot.Node(to_branch2, label = str(nprice2)); graph.add_node(node)

                counter += 1
                branch += 1

        option_price = round(option_prices[0][0],2)
        node = pydot.Node("0", label = str(option_price)); graph.add_node(node)
        graph.write_png(str(s0) + "_call_option.png",)
        
    return option_price

In [3]:
crr_call(s0 = 100, k = 92, rate = 0.06, sigma = 0.23, n_periods = 4)

44.710000000000001

![call](100_call_option.png)